### Step 1 - Decide in which city the company should be based

Decision should be based on: 
1. Number of Design Companies
2. Number of 


#### Libraries

In [77]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
import pandas as pd
import numpy as np

In [2]:
db = client.get_database('Ironhack')

In [5]:
c = db.get_collection('companies_collection')
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies_collection')

Designers (23%) and Developers (17%) compose 40% of the workforce. 

The company has a higher chance to recruit and retain good talent in those areas if they are located in a place that is either:
1. near design companies <br> 
OR
2. near successful tech startups that have raised more tham 1M 

In [97]:
# FIND DESIGN COMPANIES:
query_design = {"tag_list" : {'$regex':'.*design.*'}}


#FIND TECH STARTUPS:
tech_1 = {'total_money_raised':{'$regex':'M'}}
tech_2 = {'founded_year': {'$lte': 2015}}
tech_3 = {"category_code" : {'$regex':'tech'}}
query_tech = {'$and': [
    tech_1, tech_2, tech_3
]}
projection = {'_id': 0, 'name': 1, 'total_money_raised' : 1, 'offices.country_code' : 1, 'offices.city' : 1, 'offices.latitude': 1, 'offices.longitude': 1}

query_all = {'$or': [query_design, query_tech]}

companies_filtered = list(c.find(query_all, projection))
len(companies_filtered)
#res


956

In [149]:
companies_filtered

[{'name': 'Digg',
  'total_money_raised': '$45M',
  'offices': [{'city': 'San Francisco',
    'country_code': 'USA',
    'latitude': 37.764726,
    'longitude': -122.394523}]},
 {'name': 'Adobe Systems',
  'total_money_raised': '$0',
  'offices': [{'city': 'San Jose',
    'country_code': 'USA',
    'latitude': 37.320052,
    'longitude': -121.877636}]},
 {'name': 'Negonation',
  'total_money_raised': '$0',
  'offices': [{'city': '',
    'country_code': 'ESP',
    'latitude': 40.463667,
    'longitude': -3.74922}]},
 {'name': 'GENWI',
  'total_money_raised': '$7.1M',
  'offices': [{'city': 'Los Altos',
    'country_code': 'USA',
    'latitude': 33.8171,
    'longitude': -111.9035}]},
 {'name': 'Aviary',
  'total_money_raised': '$19M',
  'offices': [{'city': 'NY',
    'country_code': 'USA',
    'latitude': None,
    'longitude': None}]},
 {'name': 'BrightQube',
  'total_money_raised': '$850k',
  'offices': [{'city': 'Carlsbad',
    'country_code': 'USA',
    'latitude': 33.09568,
    'lo

In [99]:
lat = companies_filtered[0]['offices'][0]['latitude']
lon = companies_filtered[0]['offices'][0]['longitude']
country = companies_filtered[0]['offices'][0]['country_code']

lat, lon, country


(37.764726, -122.394523, 'USA')

In [110]:
country = companies_filtered[0]['offices']
country

[{'city': 'San Francisco',
  'country_code': 'USA',
  'latitude': 37.764726,
  'longitude': -122.394523}]

In [150]:
df = pd.DataFrame(companies_filtered)

df.isna().sum()

name                  0
total_money_raised    0
offices               0
dtype: int64

In [132]:
import numpy as np

In [133]:
def explode_columns(list_):
    try:
        city = list_[0]["city"]
        country_code = list_[0]["country_code"]
        latitude = list_[0]["latitude"]
        longitude = list_[0]["longitude"]
        return city, country_code, latitude, longitude
    except:
        print(list_)
        return np.nan


In [134]:
explode_columns(df['offices'][0])

('San Francisco', 'USA', 37.764726, -122.394523)

In [146]:
new_list = []

for index, row in df.iterrows():
    if len(row['offices']) != 1:
        pass
    else:
        new_list.append(row['offices'])
len(new_list)

749

In [148]:
df.shape[0] - len(new_list)

207

In [135]:
df[['city', 'country_code', 'latitude', 'longitude']] = df.offices.apply(explode_columns)
df.sample()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


ValueError: Columns must be same length as key

In [ ]:
def new_df(company_list):
    name = []
    country = []
    lat = []
    lon = []

    for i in company_list:
        name.appen(i['name'])


,name,total_money_raised,offices
0,Digg,$45M,"[{'city': 'San Francisco', 'country_code': 'US..."
1,Adobe Systems,$0,"[{'city': 'San Jose', 'country_code': 'USA', '..."
2,Negonation,$0,"[{'city': '', 'country_code': 'ESP', 'latitude..."
3,GENWI,$7.1M,"[{'city': 'Los Altos', 'country_code': 'USA', ..."
4,Aviary,$19M,"[{'city': 'NY', 'country_code': 'USA', 'latitu..."


In [ ]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict